# Inner product Estimation using Faster Amplitude Estimation

In the following code, the inner product of two 2D vectors is estimated using the faster amplitude estimation. 

The next cell loads the necessary modules and defines the class used for running Faster Amplitude Estimation by submitting each quantum circuit as a job:

In [1]:
from azure.quantum import Workspace
workspace = Workspace (
    subscription_id = "799529d3-173c-4965-a9c5-b2d64bf4ab95",
    resource_group = "nt_prototyping",
    name = "quantumdemo",
    location = "westeurope"
)

from qiskit.algorithms import AmplitudeEstimation,EstimationProblem,FasterAmplitudeEstimation
from qiskit import QuantumCircuit
from qiskit.algorithms.exceptions import AlgorithmError
import numpy as np
from azure.quantum.qiskit import AzureQuantumProvider
from qiskit.utils import QuantumInstance
from qiskit.compiler import transpile
import matplotlib.pyplot as plt

provider = AzureQuantumProvider(
  resource_id="/subscriptions/799529d3-173c-4965-a9c5-b2d64bf4ab95/resourceGroups/NT_Prototyping/providers/Microsoft.Quantum/Workspaces/QuantumDemo",
  location="West Europe"
)


class ModifiedFasterAmplitudeEstimation(FasterAmplitudeEstimation):
    def _cos_estimate(self, estimation_problem, k, shots):
        if self._quantum_instance is None:
            raise AlgorithmError("Quantum instance must be set.")

        if self._quantum_instance.is_statevector:
            circuit = self.construct_circuit(estimation_problem, k, measurement=False)
            statevector = self._quantum_instance.execute(circuit).get_statevector()

            # sum over all amplitudes where the objective qubits are 1
            prob = 0
            for i, amplitude in enumerate(statevector):
                # get bitstring of objective qubits
                full_state = bin(i)[2:].zfill(circuit.num_qubits)[::-1]
                state = "".join([full_state[i] for i in estimation_problem.objective_qubits])

                # check if it is a good state
                if estimation_problem.is_good_state(state[::-1]):
                    prob = prob + np.abs(amplitude) ** 2

            cos_estimate = 1 - 2 * prob
        else:
            circuit = self.construct_circuit(estimation_problem, k, measurement=True)
            #THE FOLLOWING IS MODIFIED TO SUBMIT AS JOB:
            circuit = transpile(circuit,self.quantum_instance.backend)
            print(circuit)
            job = self._quantum_instance.backend.run(circuit, shots=shots)
            job_id = job.id()
            print("Job id", job_id)
            result = job.result()
            print(result)
            
            #self._quantum_instance.run_config.shots = shots
            #counts = self._quantum_instance.execute(circuit).get_counts()
            counts = result.get_counts(circuit)
            self._num_oracle_calls += (2 * k + 1) * shots

            good_counts = 0
            for state, count in counts.items():
                if estimation_problem.is_good_state(state):
                    good_counts += count

            cos_estimate = 1 - 2 * good_counts / shots

        return cos_estimate




Define classes for state preparation operator A and Grover operator Q.

 $\theta_1$ and  $\theta_2$ define the 2D vector vectors: $|c \rangle = \begin{pmatrix}
\cos(\theta_1)\\
\sin(\theta_1)
\end{pmatrix}$ and $|v \rangle = \begin{pmatrix}
\cos(\theta_2)\\
\sin(\theta_2)
\end{pmatrix}$:

In [2]:
class BernoulliA(QuantumCircuit):
    """A circuit representing the Bernoulli A operator."""

    def __init__(self, theta1, theta2):
        super().__init__(2)  # circuit on 2 qubit

        self.h(0)
        self.cry(theta1, 0, 1)
        self.x(0)
        self.cry(theta2, 0, 1)
        self.h(0)

class BernoulliA_trans(QuantumCircuit):
    """A circuit representing the transposed Bernoulli A operator."""

    def __init__(self, theta1, theta2):
        super().__init__(2)  # circuit on 2 qubit

        self.h(0)
        self.cry(-theta2, 0, 1)
        self.x(0)
        self.cry(-theta1, 0, 1)
        self.h(0)

class BernoulliQ(QuantumCircuit):
    """A circuit representing the Bernoulli Q operator."""

    def __init__(self,theta1, theta2):
        super().__init__(2)  # circuit on 2 qubit
        
        A = BernoulliA(theta1, theta2)
        A_trans = BernoulliA_trans(theta1,theta2)
        
        self.x(0)
        self.z(0)
        self.x(0)
        self.compose(A_trans,inplace=True)
        self.x([0,1])
        self.h(1)
        self.cx([0],1)
        self.h(1)
        self.x([0,1])
        self.compose(A,inplace=True)

Define the angles of the two vectors:  

In [3]:
theta1, theta2 = 2*np.pi/6,0
A = BernoulliA(theta1,theta2)
Q = BernoulliQ(theta1,theta2)


Set the faster amplitude estimation solver. delta is the probability of having an estimate which is outside the error bounds of the algorithm which is determined by maxiter.  

In [4]:
backend = provider.get_backend("ionq.simulator")
quantum_instance = QuantumInstance(backend)
fae = ModifiedFasterAmplitudeEstimation(
      delta=0.01,  # target accuracy
      maxiter=2,  # determines the maximal power of the Grover operator
      quantum_instance=quantum_instance,
      rescale=True
)
problem = EstimationProblem(
    state_preparation=A,  # A operator
    grover_operator=Q,  # Q operator
    objective_qubits=[0],  # the "good" state Psi1 is identified as measuring |1> in qubit 0
)



In [5]:
fae_result = fae.estimate(problem)
#ae_result = fae.estimate(problem)
ip = np.cos(theta1/2)*np.cos(theta2/2)+np.sin(theta1/2)*np.sin(theta2/2)
ie = 1-2*fae_result.estimation
#Compare expected result with obtained result
print('Inner product estimate: {es:.5f}'.format(es = ie))
print('Expected result       : {es:.5f}'.format(es = ip))
print('Deviation             : {es:.5f}'.format(es = (ip-ie)/(ip)))


/usr/local/lib/python3.7/site-packages/qiskit/algorithms/amplitude_estimators/estimation_problem.py:194: UserWarning: Rescaling discards the Grover operator.
  warnings.warn("Rescaling discards the Grover operator.")


global phase: π/2
           ┌───┐                ┌───┐         ┌───┐       ┌───┐   »
 q_0: ─────┤ H ├──────────■─────┤ X ├────■────┤ H ├──■────┤ H ├───»
           └───┘     ┌────┴────┐└───┘┌───┴───┐└───┘  │    └───┘   »
 q_1: ───────────────┤ Ry(π/3) ├─────┤ Ry(0) ├───────┼────────────»
      ┌─────────────┐└──┬───┬──┘     └───────┘     ┌─┴─┐┌────────┐»
 q_2: ┤ Ry(0.50536) ├───┤ H ├──────────────────────┤ X ├┤ Rz(-π) ├»
      └─────────────┘   └───┘                      └───┘└────────┘»
c0: 2/════════════════════════════════════════════════════════════»
                                                                  »
«                      ┌───┐            ┌─────────┐      ┌──────────┐ »
« q_0: ───────■────────┤ X ├─────■──────┤ Ry(π/2) ├──■───┤ Ry(-π/2) ├─»
«         ┌───┴───┐    └───┘┌────┴─────┐└──┬───┬──┘  │   └──┬───┬───┘ »
« q_1: ───┤ Ry(0) ├─────────┤ Ry(-π/3) ├───┤ X ├─────■──────┤ X ├─────»
«      ┌──┴───────┴───┐     └──────────┘   └───┘   ┌─┴─┐┌───┴───┴────┐»
« q_2: ┤ R